In [158]:
#import
import os
import numpy as np
import pandas as pd
import re
import itertools
from collections import Counter
import pickle
import jieba
import matplotlib.pyplot as plt
import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
from pprint import pprint

# some analysis on dataframe of texts from nCovMemory

In [3]:
df_texts = pd.read_csv('df_texts_with_index.csv') #reading the dataframe with texts

In [4]:
df_texts['character_count'] = df_texts['texts'].str.len() #add character count per articles
df_texts

,id,texts,character_count
0,1,个临时发往武汉的口罩\n个临时发往武汉的口罩个临时发往武汉的口罩个临时发往武汉的口罩吴呈杰报...,4150
1,2,武汉隔离疫区信息孤岛与一辆鄂车的漂流\n武汉隔离疫区信息孤岛与一辆鄂车的漂流武汉隔离疫区信息...,5912
2,3,我家离华南海鲜市场很近返乡封城过年一位武汉大学生的过去一周\n我家离华南海鲜市场很近返乡封城...,2430
3,4,疫情危机中不被看见的人们武汉周边城市百姓的自救行动\n疫情危机中不被看见的人们武汉周边城市百...,8258
4,5,孝感前线医生武汉更难我们下面不好意思提要求\n孝感前线医生武汉更难我们下面不好意思提要求孝感...,4109
5,6,恐慌很正常当疫情结束才是心理问题高发期\n恐慌很正常当疫情结束才是心理问题高发期恐慌很正常当...,5115
6,7,外卖骑手出租车司机快递员志愿者封城九日他们维护着武汉的血脉\n外卖骑手出租车司机快递员志愿者...,5586
7,8,疫情中的心理援助寻找局部的安全\n疫情中的心理援助寻找局部的安全疫情中的心理援助寻找局部的安...,5707
8,9,鸡在瘟疫蔓延时\n鸡在瘟疫蔓延时鸡在瘟疫蔓延时鸡在瘟疫蔓延时蔚琳颖迪呈杰敏洁报道报道记录人物...,6193
9,10,武汉封城后的宠物营救计划人类犯错的后果不该由它们承担\n武汉封城后的宠物营救计划人类犯错的后...,4668


In [5]:
df_texts.sort_values('character_count', ascending = True)

,id,texts,character_count
4280,4317,保持距离无人生还\n保持距离无人生还保持距离无人生还保持距离无人生还水妈人是最美的风景人是最...,77
913,927,标题武汉情况很难说正在变好重症确诊真的收治完成了吗以及根据丁香医生实时数据截至年月日时全国累...,88
3095,3121,出天门记\n出天门记出天门记出天门记水妈人是最美的风景人是最美的风景人是万物的尺度人是最美的...,96
3885,3922,不明白\n不明白不明白不明白水妈人是最美的风景人是最美的风景人是万物的尺度人是最美的风景年月...,114
3234,3261,留言版发的人\n留言版发的人留言版发的人留言版发的人人物人是最美的风景人是最美的风景人是万物...,135
3078,3102,控制灵魂对自由的渴望\n控制灵魂对自由的渴望控制灵魂对自由的渴望控制灵魂对自由的渴望水妈人是...,175
1886,1903,我不明白\n我不明白我不明白我不明白世风世风重新认识世界我不明白我不明白一场不必要的战争他们...,176
4270,4307,武大人民医院一消化科医生感染新冠曾接诊无症状感染者更新政经频道财新网\n武大人民医院一消化科...,190
3077,3101,青山一道同担风雨\n青山一道同担风雨青山一道同担风雨青山一道同担风雨水妈人是最美的风景人是最...,194
2011,2028,文旅部提醒切勿前往美国旅游\n文旅部提醒切勿前往美国旅游文旅部提醒切勿前往美国旅游文旅部提醒...,221


In [6]:
df_texts.character_count.sum(axis=0) #total character length

15378719

In [7]:
df_texts.character_count.mean(axis=0) #mean of articles character length

3488.8200998185116

In [8]:
data = pd.read_csv('data.csv') #reading the dataframe with info (link, title, media,etc)
data.head()

,id,category,update,media,date,title,title_en,url,translation_en,is_deleted,alternative,archive
0,4444,non_fiction,2020-03-26,所有的鱼,2020-03-27,冷感、忧虑、支持，我们面对个人信息收集的态度是什么？（上）,NaN,https://mp.weixin.qq.com/s/2h4NeBv2qAt1JSiIF2PICg,NaN,NaN,NaN,https://archive.li/wip/kRknx
1,4443,non_fiction,2020-03-26,荷戟周刊,2020-03-22,武汉封城60天：那些黑暗中的微光，以及低沉的呼喊,NaN,https://mp.weixin.qq.com/s/sDI51moz5LM2yxYfgipbfA,NaN,NaN,NaN,http://archive.ph/3TI7x
2,4442,non_fiction,2020-03-26,荷戟周刊,2020-03-01,知情者口述：盛会，病人，难以逾越的冬天,NaN,https://mp.weixin.qq.com/s/k0HELMTysMTZyF7ECuDKSA,NaN,NaN,NaN,http://archive.ph/5xQgW
3,4441,non_fiction,2020-03-26,荷戟周刊,2020-03-03,患者口述：火车站，水果刀，与世界上最好喝的稀粥,NaN,https://mp.weixin.qq.com/s/HsYLaKOY8bET_NPrAdDC8A,NaN,NaN,NaN,http://archive.vn/HZPSY
4,4440,non_fiction,2020-03-26,荷戟周刊,2020-03-26,幸存者口述：那些领取榾咴以及购买墓哋的人们,NaN,https://mp.weixin.qq.com/s/0-cVgp2qE5Bpl5D4kt5aQQ,NaN,NaN,NaN,https://archive.li/wip/4l0mU
5,4439,narrative,2020-03-26,PlantifulSoul,2020-03-26,疫情日记 | 武汉封城的第六十四天,NaN,https://mp.weixin.qq.com/s/EKwzTf1Q4qrk0GPVOagNMg,NaN,NaN,NaN,http://archive.ph/ft7Bc
6,4438,narrative,2020-03-26,叶青,2020-03-25,叶青武汉日记：全国每周都可以休息2.5天吗？,NaN,http://mp.zhisland.com/fe/router/news/detail/2...,NaN,NaN,NaN,http://archive.ph/Zxtmr
7,4437,narrative,2020-03-26,叶青,2020-03-26,叶青武汉日记：把疫苗产业放在武汉？,NaN,http://mp.zhisland.com/fe/router/news/detail/2...,NaN,NaN,NaN,http://archive.ph/NLtas
8,4436,non_fiction,2020-03-26,优良better,2020-03-26,我和破产之间，只隔了一次疫情的距离,NaN,https://mp.weixin.qq.com/s/wIXf9Z2bjECHR95zr0Z8kA,NaN,NaN,NaN,http://archive.ph/xOOVH
9,4435,non_fiction,2020-03-26,新民周刊,2020-03-26,口述实录 | 天价机票、36小时只吃一顿饭，滞留英国的13岁女儿终于回国了,NaN,https://mp.weixin.qq.com/s/9wSD5UVKXpGlZw_nE8Raag,NaN,NaN,NaN,http://archive.ph/bCcCM


In [151]:
with open('UsrWords.txt', 'r', encoding='utf8') as f:
    custom_noun = f.readlines()
    for noun in custom_noun:
        jieba.add_word(noun.replace('\n', ''), freq=None, tag='n')

In [152]:
df_texts['words'] = df_texts.texts.apply(jieba.lcut)
df_texts.head()

id  \
0  1    
1  2    
2  3    
3  4    
4  5    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [153]:
fh = open("stopwords.txt")
stopwords=[]
stopwords.append('\n') #append the '\n' in the beginning otherwise it would be erased by strip
stopwords.append('%')  #we append some special characters, I am also goign to add it to the cleaning function pre-processing
stopwords.append('|')  #the texts but sometimes it does not catch them all
stopwords.append('△')
stopwords.append('▼')
stopwords.append('｜')
stopwords.append('\u3000') #this is a special empty space, I also removed tab space in the cleaning function, perhaps that'll help too
for line in fh:
    line = line.rstrip()
    stopwords.append(line)

def stp_removal(wordlist, stopwords=stopwords):
    lili=[]
    for word in wordlist:
        if word not in stopwords:
            lili.append(word)
    return lili

df_texts['words'] = df_texts.words.apply(stp_removal)
df_texts.head()

id  \
0  1    
1  2    
2  3    
3  4    
4  5    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [177]:
ldamallet = pickle.load(open("ldamallet.pkl", "rb")) #loading the model with pickle


In [178]:
import os
from gensim.models.wrappers import LdaMallet

os.environ.update({'MALLET_HOME':r'/home/bilatli/DEV/mallet-2.0.8/'}) 
#You should update this path as per the path of Mallet directory on your system.
mallet_path = r'/home/bilatli/DEV/mallet-2.0.8/bin/mallet.bat' 
#You should update this path as per the path of Mallet directory on your system.

In [176]:
data_lemmatized = df_texts.words

id2word = gensim.corpora.Dictionary(data_lemmatized)

id2word.filter_extremes(no_above= 0.29) 

corpus = [id2word.doc2bow(doc) for doc in data_lemmatized]


In [179]:
df_result = ldamallet[corpus]

In [197]:
corpus_topics = [sorted(topics, key=lambda record: -record[1])[0] for topics in df_result]

topics = [[(term, round(wt, 3)) for term, wt in lda.show_topic(n, topn=20)] for n in range(0, lda.num_topics)]

# showing terms of topics
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame([', '.join([term for term, wt in topic]) for topic in topics], columns = ['Terms per Topic'], index=['Topic'+str(t) for t in range(1, lda.num_topics+1)] )
topics_df

,Terms per Topic
Topic1,"孩子, 出门, 妈妈, 回来, 小区, 母亲, 回, 超市, 东西, 群里, 手机, 开, 打电话, 过年, 父亲, 父母, 早上, 跑, 爸爸, 志愿者"
Topic2,"护士, 发热, 方舱, 病房, 床位, 收治, 门诊, 防护服, 出院, 医疗队, 疑似, 家属, 核酸, 同事, 病情, 救治, 住院, 病区, 定点医院, 科室"
Topic3,"回复, 日记, 老师, 学生, 心理, 游戏, 写, 学校, 情绪, 学习, 微博, 记录, 举报, 网络, 面对, 喜欢, 相信, 灾难, 直播, 网友"
Topic4,"美国, 意大利, 韩国, 日本, 英国, 伊朗, 航班, 国内, 欧洲, 德国, 累计, 乘客, 回国, 机场, 入境, 关闭, 香港, 取消, 邮轮, 特朗普"
Topic5,"临床, 疫苗, 传染病, 疾病, 药物, 疾控中心, 感染者, 公共卫生, 核酸, 卫健委, 诊断, 团队, 有效, 野生动物, 方案, 华南, 研发, 卫生, 实验室, 系统"
Topic6,"评论, 登录, 代表, 观点, 金融, 博客, 仅, 政经, 投资, 微博, 复工, 网友, 最新, 科技, 课, 发表, 个人观点, 会议, 数字, 私房"
Topic7,"捐赠, 湖北省, 志愿者, 居民, 酒店, 小区, 医用, 指挥部, 累计, 保障, 单位, 防护服, 排查, 红十字会, 司机, 车辆, 采购, 黄冈, 部门, 监狱"
Topic8,"复工, 生产, 员工, 行业, 需求, 产品, 政策, 工厂, 销售, 价格, 订单, 业务, 办公, 收入, 成本, 消费, 恢复, 面临, 资金, 客户"


In [198]:
# create a dataframe
corpus_topic_df = pd.DataFrame()

# get the Titles from the original dataframe
corpus_topic_df['id'] = df_texts.id

corpus_topic_df['Dominant_Topic'] = [item[0]+1 for item in corpus_topics]
corpus_topic_df['Contribution %'] = [round(item[1]*100, 2) for item in corpus_topics]
corpus_topic_df['Topic_Terms'] = [topics_df.iloc[t[0]]['Terms per Topic'] for t in corpus_topics]

corpus_topic_df.head()

,id,Dominant_Topic,Contribution %,Topic_Terms
0,1,2,46.63,"护士, 发热, 方舱, 病房, 床位, 收治, 门诊, 防护服, 出院, 医疗队, 疑似, 家属, 核酸, 同事, 病情, 救治, 住院, 病区, 定点医院, 科室"
1,2,2,52.84,"护士, 发热, 方舱, 病房, 床位, 收治, 门诊, 防护服, 出院, 医疗队, 疑似, 家属, 核酸, 同事, 病情, 救治, 住院, 病区, 定点医院, 科室"
2,3,7,57.89,"捐赠, 湖北省, 志愿者, 居民, 酒店, 小区, 医用, 指挥部, 累计, 保障, 单位, 防护服, 排查, 红十字会, 司机, 车辆, 采购, 黄冈, 部门, 监狱"
3,4,7,32.04,"捐赠, 湖北省, 志愿者, 居民, 酒店, 小区, 医用, 指挥部, 累计, 保障, 单位, 防护服, 排查, 红十字会, 司机, 车辆, 采购, 黄冈, 部门, 监狱"
4,5,2,42.83,"护士, 发热, 方舱, 病房, 床位, 收治, 门诊, 防护服, 出院, 医疗队, 疑似, 家属, 核酸, 同事, 病情, 救治, 住院, 病区, 定点医院, 科室"


In [203]:
dominant_topic_df = pd.DataFrame()

xx = corpus_topic_df.groupby(by='Dominant_Topic').size()
dominant_topic_df['Doc_Count'] = xx
dominant_topic_df['Total_Docs_Perc'] = xx
dominant_topic_df['Total_Docs_Perc'] = dominant_topic_df['Total_Docs_Perc'].apply(lambda row: round((row*100) / len(corpus), 2))


dominant_topic_df

,Doc_Count,Total_Docs_Perc
Dominant_Topic,,
1,532,12.07
2,453,10.28
3,613,13.91
4,559,12.68
5,406,9.21
6,317,7.19
7,1125,25.52
8,403,9.14


In [204]:
dominant_topic_df.Doc_Count.sum()

4408

## Non-fiction articles

In [194]:
nf = data[data.category == 'non_fiction'] #susbset by category first non-fiction
nf

,id,category,update,media,date,title,title_en,url,translation_en,is_deleted,alternative,archive
0,4444,non_fiction,2020-03-26,所有的鱼,2020-03-27,冷感、忧虑、支持，我们面对个人信息收集的态度是什么？（上）,NaN,https://mp.weixin.qq.com/s/2h4NeBv2qAt1JSiIF2PICg,NaN,NaN,NaN,https://archive.li/wip/kRknx
1,4443,non_fiction,2020-03-26,荷戟周刊,2020-03-22,武汉封城60天：那些黑暗中的微光，以及低沉的呼喊,NaN,https://mp.weixin.qq.com/s/sDI51moz5LM2yxYfgipbfA,NaN,NaN,NaN,http://archive.ph/3TI7x
2,4442,non_fiction,2020-03-26,荷戟周刊,2020-03-01,知情者口述：盛会，病人，难以逾越的冬天,NaN,https://mp.weixin.qq.com/s/k0HELMTysMTZyF7ECuDKSA,NaN,NaN,NaN,http://archive.ph/5xQgW
3,4441,non_fiction,2020-03-26,荷戟周刊,2020-03-03,患者口述：火车站，水果刀，与世界上最好喝的稀粥,NaN,https://mp.weixin.qq.com/s/HsYLaKOY8bET_NPrAdDC8A,NaN,NaN,NaN,http://archive.vn/HZPSY
4,4440,non_fiction,2020-03-26,荷戟周刊,2020-03-26,幸存者口述：那些领取榾咴以及购买墓哋的人们,NaN,https://mp.weixin.qq.com/s/0-cVgp2qE5Bpl5D4kt5aQQ,NaN,NaN,NaN,https://archive.li/wip/4l0mU
8,4436,non_fiction,2020-03-26,优良better,2020-03-26,我和破产之间，只隔了一次疫情的距离,NaN,https://mp.weixin.qq.com/s/wIXf9Z2bjECHR95zr0Z8kA,NaN,NaN,NaN,http://archive.ph/xOOVH
9,4435,non_fiction,2020-03-26,新民周刊,2020-03-26,口述实录 | 天价机票、36小时只吃一顿饭，滞留英国的13岁女儿终于回国了,NaN,https://mp.weixin.qq.com/s/9wSD5UVKXpGlZw_nE8Raag,NaN,NaN,NaN,http://archive.ph/bCcCM
10,4434,non_fiction,2020-03-26,新民周刊,2020-03-26,口述实录 | 煎熬！在法国，我被房东赶出家门；回国后，我坐上了救护车,NaN,https://mp.weixin.qq.com/s/B2xJ_ejAZIimpuYLO9WWDQ,NaN,NaN,NaN,http://archive.ph/oqRPs
11,4433,non_fiction,2020-03-26,新民周刊,2020-03-26,上海作为“第一入境点”后，100%采样检测，海关揭秘如何针对性检疫防控,NaN,https://mp.weixin.qq.com/s/4zCvRJ7u-4ke9InrjLWkpw,NaN,NaN,NaN,http://archive.ph/Thvih
12,4432,non_fiction,2020-03-26,DeepTech深科技,2020-03-26,新冠疫情下美国的“救命稻草”，或源于麻省理工学院十年前的一个课程设计,NaN,https://mp.weixin.qq.com/s/rk9hMJ_KrwdMO3E3SbWAFQ,NaN,NaN,NaN,http://archive.ph/56auX


In [10]:
nf_texts = df_texts.loc[df_texts['id'].isin(nf.id)] #creating new dataframe with only non-fiction texts
nf_texts

,id,texts,character_count
0,1,个临时发往武汉的口罩\n个临时发往武汉的口罩个临时发往武汉的口罩个临时发往武汉的口罩吴呈杰报...,4150
1,2,武汉隔离疫区信息孤岛与一辆鄂车的漂流\n武汉隔离疫区信息孤岛与一辆鄂车的漂流武汉隔离疫区信息...,5912
2,3,我家离华南海鲜市场很近返乡封城过年一位武汉大学生的过去一周\n我家离华南海鲜市场很近返乡封城...,2430
3,4,疫情危机中不被看见的人们武汉周边城市百姓的自救行动\n疫情危机中不被看见的人们武汉周边城市百...,8258
4,5,孝感前线医生武汉更难我们下面不好意思提要求\n孝感前线医生武汉更难我们下面不好意思提要求孝感...,4109
5,6,恐慌很正常当疫情结束才是心理问题高发期\n恐慌很正常当疫情结束才是心理问题高发期恐慌很正常当...,5115
6,7,外卖骑手出租车司机快递员志愿者封城九日他们维护着武汉的血脉\n外卖骑手出租车司机快递员志愿者...,5586
7,8,疫情中的心理援助寻找局部的安全\n疫情中的心理援助寻找局部的安全疫情中的心理援助寻找局部的安...,5707
8,9,鸡在瘟疫蔓延时\n鸡在瘟疫蔓延时鸡在瘟疫蔓延时鸡在瘟疫蔓延时蔚琳颖迪呈杰敏洁报道报道记录人物...,6193
9,10,武汉封城后的宠物营救计划人类犯错的后果不该由它们承担\n武汉封城后的宠物营救计划人类犯错的后...,4668


In [185]:
print('character stats of non-fiction category:')

print('total character length:', nf_texts.character_count.sum(axis=0)) #sum

print('min character length:', nf_texts.character_count.min(axis=0)) #min

print('max character length:', nf_texts.character_count.max(axis=0)) #max

print('mean character length:', nf_texts.character_count.mean(axis=0)) #median

print('median character length:', nf_texts.character_count.median(axis=0)) #median

character stats of non-fiction category:
total character length: 12335008
min character length: 88
max character length: 64525
mean character length: 3580.5538461538463
median character length: 3274.0


In [14]:
nf_texts[nf_texts.character_count == 64525] #check which article is the longest of non-fiction

,id,texts,character_count
3254,3281,武汉新型冠状病毒肺炎大事记年月日今财新网财新网\n武汉新型冠状病毒肺炎大事记年月日今财新网财...,64525


In [15]:
nf_texts[nf_texts.character_count == 88] #check which article is the shortest of non-fiction
#an article showed only 1 character, we then went back to cleaning and cambe back here

,id,texts,character_count
913,927,标题武汉情况很难说正在变好重症确诊真的收治完成了吗以及根据丁香医生实时数据截至年月日时全国累...,88


In [199]:
nf_dom_texts = corpus_topic_df.loc[df_texts['id'].isin(nf.id)] #subsetting non-fiction texts

nf_dom_texts

,id,Dominant_Topic,Contribution %,Topic_Terms
0,1,2,46.63,"护士, 发热, 方舱, 病房, 床位, 收治, 门诊, 防护服, 出院, 医疗队, 疑似, 家属, 核酸, 同事, 病情, 救治, 住院, 病区, 定点医院, 科室"
1,2,2,52.84,"护士, 发热, 方舱, 病房, 床位, 收治, 门诊, 防护服, 出院, 医疗队, 疑似, 家属, 核酸, 同事, 病情, 救治, 住院, 病区, 定点医院, 科室"
2,3,7,57.89,"捐赠, 湖北省, 志愿者, 居民, 酒店, 小区, 医用, 指挥部, 累计, 保障, 单位, 防护服, 排查, 红十字会, 司机, 车辆, 采购, 黄冈, 部门, 监狱"
3,4,7,32.04,"捐赠, 湖北省, 志愿者, 居民, 酒店, 小区, 医用, 指挥部, 累计, 保障, 单位, 防护服, 排查, 红十字会, 司机, 车辆, 采购, 黄冈, 部门, 监狱"
4,5,2,42.83,"护士, 发热, 方舱, 病房, 床位, 收治, 门诊, 防护服, 出院, 医疗队, 疑似, 家属, 核酸, 同事, 病情, 救治, 住院, 病区, 定点医院, 科室"
5,6,4,41.94,"美国, 意大利, 韩国, 日本, 英国, 伊朗, 航班, 国内, 欧洲, 德国, 累计, 乘客, 回国, 机场, 入境, 关闭, 香港, 取消, 邮轮, 特朗普"
6,7,2,58.39,"护士, 发热, 方舱, 病房, 床位, 收治, 门诊, 防护服, 出院, 医疗队, 疑似, 家属, 核酸, 同事, 病情, 救治, 住院, 病区, 定点医院, 科室"
7,8,4,35.24,"美国, 意大利, 韩国, 日本, 英国, 伊朗, 航班, 国内, 欧洲, 德国, 累计, 乘客, 回国, 机场, 入境, 关闭, 香港, 取消, 邮轮, 特朗普"
8,9,2,50.29,"护士, 发热, 方舱, 病房, 床位, 收治, 门诊, 防护服, 出院, 医疗队, 疑似, 家属, 核酸, 同事, 病情, 救治, 住院, 病区, 定点医院, 科室"
9,10,7,59.56,"捐赠, 湖北省, 志愿者, 居民, 酒店, 小区, 医用, 指挥部, 累计, 保障, 单位, 防护服, 排查, 红十字会, 司机, 车辆, 采购, 黄冈, 部门, 监狱"


In [200]:
nf_dominant_topic_df = pd.DataFrame()

xx = nf_dom_texts.groupby(by='Dominant_Topic').size()
nf_dominant_topic_df['Doc_Count'] = xx
nf_dominant_topic_df['Total_Docs_Perc'] = xx
nf_dominant_topic_df['Total_Docs_Perc'] = nf_dominant_topic_df['Total_Docs_Perc'].apply(lambda row: round((row*100) / len(corpus), 2))


nf_dominant_topic_df

,Doc_Count,Total_Docs_Perc
Dominant_Topic,,
1,527,11.96
2,405,9.19
3,548,12.43
4,529,12.00
5,332,7.53
6,315,7.15
7,404,9.17
8,385,8.73


## Narrative

Subsetting the by the narrative category

In [147]:
df_narrative = data[data.category == 'narrative'] #susbset by category narrative
#df_narrative.head()

In [189]:
narrative_texts = df_texts.loc[df_texts['id'].isin(df_narrative.id)] #subsetting narrative texts 
narrative_texts

id  \
225   226    
226   227    
227   228    
384   391    
385   392    
386   393    
387   394    
388   395    
389   396    
390   397    
391   398    
392   399    
503   510    
504   511    
505   512    
506   513    
507   514    
508   515    
509   516    
510   517    
511   518    
512   519    
513   520    
514   521    
515   522    
516   523    
517   524    
518   525    
519   526    
520   527    
...   ...    
4321  4358   
4322  4359   
4324  4361   
4325  4362   
4326  4363   
4327  4364   
4330  4367   
4335  4372   
4336  4373   
4337  4374   
4338  4375   
4339  4376   
4340  4377   
4341  4378   
4352  4389   
4353  4390   
4356  4393   
4371  4408   
4381  4418   
4382  4419   
4383  4420   
4384  4421   
4385  4422   
4386  4423   
4387  4424   
4388  4425   
4392  4429   
4400  4437   
4401  4438   
4402  4439   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

Character stats of narrative category:

In [149]:
print('character stats of narrative category:')

print('total character length:', narrative_texts.character_count.sum(axis=0)) #sum

print('min character length:', narrative_texts.character_count.min(axis=0)) #min

print('max character length:', narrative_texts.character_count.max(axis=0)) #max

print('mean character length:', narrative_texts.character_count.mean(axis=0)) #mean

print('median character length:', narrative_texts.character_count.median(axis=0)) #median

character stats of narrative category:
total character length: 3043711
min character length: 77
max character length: 12043
mean character length: 3160.6552440290757
median character length: 2905.0


In [201]:
n_dom_texts = corpus_topic_df.loc[df_texts['id'].isin(df_narrative.id)] #subsetting narrative texts 

In [202]:
dominant_topic_df = pd.DataFrame()

xx = n_dom_texts.groupby(by='Dominant_Topic').size()
dominant_topic_df['Doc_Count'] = xx
dominant_topic_df['Total_Docs_Perc'] = xx
dominant_topic_df['Total_Docs_Perc'] = dominant_topic_df['Total_Docs_Perc'].apply(lambda row: round((row*100) / len(corpus), 2))


dominant_topic_df

,Doc_Count,Total_Docs_Perc
Dominant_Topic,,
1,5,0.11
2,48,1.09
3,65,1.47
4,30,0.68
5,74,1.68
6,2,0.05
7,721,16.36
8,18,0.41


Applying our previously created lda model separately

In [28]:
with open('UsrWords.txt', 'r', encoding='utf8') as f:
    custom_noun = f.readlines()
    for noun in custom_noun:
        jieba.add_word(noun.replace('\n', ''), freq=None, tag='n')

In [29]:
narrative_texts['words'] = narrative_texts.texts.apply(jieba.lcut)
narrative_texts.head()

/home/bilatli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,id,texts,character_count,words
225,226,五个武汉人的封城日记故事\n五个武汉人的封城日记故事五个武汉人的封城日记故事五个武汉人的封城...,6399,"[五个, 武汉, 人, 的, 封城, 日记, 故事, \n, 五个, 武汉, 人, 的, 封..."
226,227,武汉医生工作见闻下了班我还得帮科室抢物资故事\n武汉医生工作见闻下了班我还得帮科室抢物资故事...,3024,"[武汉, 医生, 工作, 见闻, 下, 了, 班, 我, 还, 得, 帮, 科室, 抢, 物..."
227,228,三个武汉人的外地隔离日记故事\n三个武汉人的外地隔离日记故事三个武汉人的外地隔离日记故事三个...,7346,"[三个, 武汉, 人, 的, 外地, 隔离, 日记, 故事, \n, 三个, 武汉, 人, ..."
384,391,预产期还有天我在武汉待产的医院被征用武汉日常每日书\n预产期还有天我在武汉待产的医院被征用武...,2661,"[预产期, 还有, 天, 我, 在, 武汉, 待产, 的, 医院, 被, 征用, 武汉, 日..."
385,392,不忘故乡因疫情严重禁止一切人员车辆出入三明治武汉每日书\n不忘故乡因疫情严重禁止一切人员车辆...,5865,"[不, 忘, 故乡, 因, 疫情, 严重, 禁止, 一切, 人员, 车辆, 出入, 三明治,..."


In [30]:
fh = open("stopwords.txt")
stopwords=[]
stopwords.append('\n') #append the '\n' in the beginning otherwise it would be erased by strip
stopwords.append('%')  #we append some special characters, I am also goign to add it to the cleaning function pre-processing
stopwords.append('|')  #the texts but sometimes it does not catch them all
stopwords.append('△')
stopwords.append('▼')
stopwords.append('｜')
stopwords.append('\u3000') #this is a special empty space, I also removed tab space in the cleaning function, perhaps that'll help too
for line in fh:
    line = line.rstrip()
    stopwords.append(line)

def stp_removal(wordlist, stopwords=stopwords):
    lili=[]
    for word in wordlist:
        if word not in stopwords:
            lili.append(word)
    return lili

narrative_texts['words'] = narrative_texts.words.apply(stp_removal)
narrative_texts.head()

/home/bilatli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,texts,character_count,words
225,226,五个武汉人的封城日记故事\n五个武汉人的封城日记故事五个武汉人的封城日记故事五个武汉人的封城...,6399,"[五个, 武汉, 封城, 日记, 五个, 武汉, 封城, 日记, 五个, 武汉, 封城, 日..."
226,227,武汉医生工作见闻下了班我还得帮科室抢物资故事\n武汉医生工作见闻下了班我还得帮科室抢物资故事...,3024,"[武汉, 医生, 工作, 见闻, 班, 帮, 科室, 抢, 物资, 武汉, 医生, 工作, ..."
227,228,三个武汉人的外地隔离日记故事\n三个武汉人的外地隔离日记故事三个武汉人的外地隔离日记故事三个...,7346,"[三个, 武汉, 外地, 隔离, 日记, 三个, 武汉, 外地, 隔离, 日记, 三个, 武..."
384,391,预产期还有天我在武汉待产的医院被征用武汉日常每日书\n预产期还有天我在武汉待产的医院被征用武...,2661,"[预产期, 天, 武汉, 待产, 医院, 征用, 武汉, 日常, 每日, 书, 预产期, 天..."
385,392,不忘故乡因疫情严重禁止一切人员车辆出入三明治武汉每日书\n不忘故乡因疫情严重禁止一切人员车辆...,5865,"[忘, 故乡, 疫情, 严重, 禁止, 人员, 车辆, 出入, 武汉, 每日, 书, 忘, ..."


In [43]:
data_lemmatized = narrative_texts.words

id2word = gensim.corpora.Dictionary(data_lemmatized)

id2word.filter_extremes(no_above= 0.29) 

corpus = [id2word.doc2bow(doc) for doc in data_lemmatized]



225     [五个, 武汉, 封城, 日记, 五个, 武汉, 封城, 日记, 五个, 武汉, 封城, 日...
226     [武汉, 医生, 工作, 见闻, 班, 帮, 科室, 抢, 物资, 武汉, 医生, 工作, ...
227     [三个, 武汉, 外地, 隔离, 日记, 三个, 武汉, 外地, 隔离, 日记, 三个, 武...
384     [预产期, 天, 武汉, 待产, 医院, 征用, 武汉, 日常, 每日, 书, 预产期, 天...
385     [忘, 故乡, 疫情, 严重, 禁止, 人员, 车辆, 出入, 武汉, 每日, 书, 忘, ...
386     [视频, 麻将, 做个, 美食, 武汉, 人宅, 解压, 法, 武汉, 每日, 书, 视频,...
387     [武汉, 除夕夜, 危机, 亲情, 祈祷, 持续, 更新, 武汉, 除夕夜, 危机, 亲情,...
388     [位, 武汉, 亲笔写, 封城, 第一天, 生活, 持续, 更新, 位, 武汉, 亲笔写, ...
389     [坐, 一辆, 确诊, 患者, 公交车, 坐, 一辆, 确诊, 患者, 公交车, 坐, 一辆...
390     [武汉, 封城后, 公司, 撑, 多久, 武汉, 封城后, 公司, 撑, 多久, 武汉, 封...
391     [黄冈, 医院, 保洁, 母亲, 举报, 黄冈, 医院, 保洁, 母亲, 举报, 黄冈, 医...
392     [封城后, 武汉, 买, 封城后, 武汉, 买, 封城后, 武汉, 买, 封城后, 武汉, ...
503     [离开, 家时, 真切感, 恐惧, 丨, 离开, 家时, 真切感, 恐惧, 丨, 离开, 家...
504     [万人, 离开, 武汉, 武汉, 日志, 万人, 离开, 武汉, 武汉, 日志, 万人, 离...
505     [武汉, 餐饮, 冬天, 春天, 丨, 武汉, 餐饮, 冬天, 春天, 丨, 武汉, 餐饮,...
506     [三千里, 自驾, 返京, 记, 丨, 三千里, 自驾, 返京, 记, 丨, 三千里, 自驾...
507     [大年三十, 父母, 送, 离开, 湖北, 老家, 丨, 大年三十, 父母, 送, 离开, ...
508     [丨, 武汉

In [45]:
n_result = lda[corpus]

In [48]:
corpus_topics = [sorted(topics, key=lambda record: -record[1])[0] for topics in n_result]

In [150]:
topics = [[(term, round(wt, 3)) for term, wt in lda.show_topic(n, topn=20)] for n in range(0, lda.num_topics)]

In [59]:
# showing terms of topics
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame([', '.join([term for term, wt in topic]) for topic in topics], columns = ['Terms per Topic'], index=['Topic'+str(t) for t in range(1, lda.num_topics+1)] )
topics_df

,Terms per Topic
Topic1,"孩子, 出门, 妈妈, 回来, 小区, 母亲, 回, 超市, 东西, 群里, 手机, 开, 打电话, 过年, 父亲, 父母, 早上, 跑, 爸爸, 志愿者"
Topic2,"护士, 发热, 方舱, 病房, 床位, 收治, 门诊, 防护服, 出院, 医疗队, 疑似, 家属, 核酸, 同事, 病情, 救治, 住院, 病区, 定点医院, 科室"
Topic3,"回复, 日记, 老师, 学生, 心理, 游戏, 写, 学校, 情绪, 学习, 微博, 记录, 举报, 网络, 面对, 喜欢, 相信, 灾难, 直播, 网友"
Topic4,"美国, 意大利, 韩国, 日本, 英国, 伊朗, 航班, 国内, 欧洲, 德国, 累计, 乘客, 回国, 机场, 入境, 关闭, 香港, 取消, 邮轮, 特朗普"
Topic5,"临床, 疫苗, 传染病, 疾病, 药物, 疾控中心, 感染者, 公共卫生, 核酸, 卫健委, 诊断, 团队, 有效, 野生动物, 方案, 华南, 研发, 卫生, 实验室, 系统"
Topic6,"评论, 登录, 代表, 观点, 金融, 博客, 仅, 政经, 投资, 微博, 复工, 网友, 最新, 科技, 课, 发表, 个人观点, 会议, 数字, 私房"
Topic7,"捐赠, 湖北省, 志愿者, 居民, 酒店, 小区, 医用, 指挥部, 累计, 保障, 单位, 防护服, 排查, 红十字会, 司机, 车辆, 采购, 黄冈, 部门, 监狱"
Topic8,"复工, 生产, 员工, 行业, 需求, 产品, 政策, 工厂, 销售, 价格, 订单, 业务, 办公, 收入, 成本, 消费, 恢复, 面临, 资金, 客户"


Defining a dominant topic per article

In [116]:
# create a dataframe
corpus_topic_df = pd.DataFrame()

# get the Titles from the original dataframe
corpus_topic_df['id'] = narrative_texts.id

corpus_topic_df['Dominant_Topic'] = [item[0]+1 for item in corpus_topics]
corpus_topic_df['Contribution %'] = [round(item[1]*100, 2) for item in corpus_topics]
corpus_topic_df['Topic_Terms'] = [topics_df.iloc[t[0]]['Terms per Topic'] for t in corpus_topics]

corpus_topic_df.head()

,id,Dominant_Topic,Contribution %,Topic_Terms
225,226,1,43.77,"孩子, 出门, 妈妈, 回来, 小区, 母亲, 回, 超市, 东西, 群里, 手机, 开, 打电话, 过年, 父亲, 父母, 早上, 跑, 爸爸, 志愿者"
226,227,1,60.47,"孩子, 出门, 妈妈, 回来, 小区, 母亲, 回, 超市, 东西, 群里, 手机, 开, 打电话, 过年, 父亲, 父母, 早上, 跑, 爸爸, 志愿者"
227,228,1,61.75,"孩子, 出门, 妈妈, 回来, 小区, 母亲, 回, 超市, 东西, 群里, 手机, 开, 打电话, 过年, 父亲, 父母, 早上, 跑, 爸爸, 志愿者"
384,391,1,55.54,"孩子, 出门, 妈妈, 回来, 小区, 母亲, 回, 超市, 东西, 群里, 手机, 开, 打电话, 过年, 父亲, 父母, 早上, 跑, 爸爸, 志愿者"
385,392,1,42.98,"孩子, 出门, 妈妈, 回来, 小区, 母亲, 回, 超市, 东西, 群里, 手机, 开, 打电话, 过年, 父亲, 父母, 早上, 跑, 爸爸, 志愿者"


Dominant topic distribution amongst texts in the narrative category: 

In [144]:
dominant_topic_df = pd.DataFrame()

xx = corpus_topic_df.groupby(by='Dominant_Topic').size()
dominant_topic_df['Doc_Count'] = xx
dominant_topic_df['Total_Docs_Perc'] = xx
dominant_topic_df['Total_Docs_Perc'] = dominant_topic_df['Total_Docs_Perc'].apply(lambda row: round((row*100) / len(corpus), 2))


dominant_topic_df

,Doc_Count,Total_Docs_Perc
Dominant_Topic,,
1,955,99.17
3,5,0.52
5,1,0.10
7,1,0.10
8,1,0.10


The result is very different than with the first solution, probably because the filtering of the words. 

In [98]:
corpus_topic_df.groupby('Dominant_Topic').apply(lambda topic_set:
(topic_set.sort_values(by=['Contribution %'], ascending=False).iloc[0])).reset_index(drop=True)

,id,Dominant Topic,Contribution %,Topic Terms
0,228,1,61.75,"孩子, 出门, 妈妈, 回来, 小区, 母亲, 回, 超市, 东西, 群里, 手机, 开, 打电话, 过年, 父亲, 父母, 早上, 跑, 爸爸, 志愿者"
1,397,3,31.30,"回复, 日记, 老师, 学生, 心理, 游戏, 写, 学校, 情绪, 学习, 微博, 记录, 举报, 网络, 面对, 喜欢, 相信, 灾难, 直播, 网友"
2,702,5,28.59,"临床, 疫苗, 传染病, 疾病, 药物, 疾控中心, 感染者, 公共卫生, 核酸, 卫健委, 诊断, 团队, 有效, 野生动物, 方案, 华南, 研发, 卫生, 实验室, 系统"
3,3117,7,20.38,"捐赠, 湖北省, 志愿者, 居民, 酒店, 小区, 医用, 指挥部, 累计, 保障, 单位, 防护服, 排查, 红十字会, 司机, 车辆, 采购, 黄冈, 部门, 监狱"
4,3261,8,15.53,"复工, 生产, 员工, 行业, 需求, 产品, 政策, 工厂, 销售, 价格, 订单, 业务, 办公, 收入, 成本, 消费, 恢复, 面临, 资金, 客户"
